In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
from scipy import stats
import re
from tqdm import tqdm
import json
import sys

import requests
# from urllib.request import urlopen
from bs4 import BeautifulSoup
# from pyquery import PyQuery 
# import asyncio
# import aiohttp
# from aiohttp import ClientSession, ClientConnectorError
# import pathlib
# imported🄬ｆｄｄｄｄｄｄｄｄｄｄｄｄｄｄｄｄｄsys
# import asyncio
# import time 
# import aiohttp
# from aiohttp.client import ClientSession
from urllib.parse import urlparse
from threading import Thread
import http.client, sys
from queue import Queue

In [2]:
books = pd.read_csv('../../../data/recommendation/Books.csv',index_col=0)
ratings = pd.read_csv('../../../data/recommendation/Ratings.csv')

print(books.shape)
books.head()

C:\Users\PAC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(271360, 7)


,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
def weighted_rating(v, R, C, m=5):
    w_r = (v/(v+m))*R + (m/(m+v))*C
    return w_r

R = ratings[['ISBN', 'Book-Rating']].groupby(['ISBN'], as_index=True).mean() # Book wise average rating
C = ratings['Book-Rating'].mean() # Average rating for all books

In [4]:
%%time
m=5
ratings = pd.concat([R, ratings.groupby(['ISBN'], as_index=True).size().to_frame()],axis=1).rename(columns={'Book-Rating':'average_rating',0:'votes'})
ratings['weighted_average'] = ratings.apply(lambda row : weighted_rating(row['votes'], row['average_rating'], C), axis = 1)

ratings.head()

Wall time: 12.6 s


,average_rating,votes,weighted_average
ISBN,,,
0330299891,3.0,2,2.904964
0375404120,1.5,2,2.476393
0586045007,0.0,1,2.389125
9022906116,3.5,2,3.047822
9032803328,0.0,1,2.389125


In [5]:
index = [idx for idx in books.index if idx in ratings.index]
df = pd.concat([books.loc[index,:], ratings.loc[index, :]], axis=1)
df = df[~df.index.duplicated(keep='first')]

df.head()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,average_rating,votes,weighted_average
ISBN,,,,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000,1,2.389125
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571,14,4.386040
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,5.000000,3,3.666844
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4.272727,11,3.833422
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0.000000,1,2.389125


In [6]:
def doWork():
    while True:
        url = q.get()
        status, url = getStatus(url)
        processResult(status, url)
        q.task_done()

def getStatus(ourl):
    try:
        url = urlparse(ourl)
        conn = httplib.HTTPConnection(url.netloc)   
        conn.request("HEAD", url.path)
        res = conn.getresponse()
        return res.status, ourl
    except:
        return "error", ourl

def processResult(status, url):
    isbn = url.split('=')[-1]
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    badges = soup.find_all("div", class_="details-ia hidden-tiles")
    res = [s.get_text().replace('\n','').rstrip() for s in badges]
    res = [s[s.find('Topic'):] for s in res]
    res = [s[s.find(':')+1:] for s in res]
    res = [[s.replace('Source: removedNEL','').lstrip().rstrip().replace('.','') for s in r.split(',')] for r in res]
    res = [i for i in list(set([i for s in res for i in s])) if i !='']
    data[isbn] = res

In [ ]:
%%time
concurrent = 200
data = {}
url_list = ['https://archive.org/details/texts?query=%s'%(isbn) for isbn in df.index]

q = Queue(concurrent * 2)
for i in range(concurrent):
    t = Thread(target=doWork)
    t.daemon = True
    t.start()
try:
    for url in tqdm(url_list):
        q.put(uuijki98orl.strip())
    q.join()
except KeyboardInterrupt:
    sys.exit(1)

 14%|█████████▍                                                            | 36540/270151 [2:23:00<14:13:19,  4.56it/s]Exception in thread Thread-100:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connection.py", line 421, in connect
    tls_in_tls=tls_in_tls,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\util\ssl_.py", line 450, in ssl_wrap_socket
    sock, context, tls_in_tls, server_hostname=server_hostname
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-pac


 92%|███████████████████████████████████████████████████████████████▍     | 248352/270151 [15:36:31<2:03:48,  2.93it/s]Exception in thread Thread-40:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connection.py", line 421, in connect
    tls_in_tls=tls_in_tls,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\util\ssl_.py", line 450, in ssl_wrap_socket
    sock, context, tls_in_tls, server_hostname=server_hostname
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-pac


Exception in thread Thread-192:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connection.py", line 421, in connect
    tls_in_tls=tls_in_tls,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\util\ssl_.py", line 450, in ssl_wrap_socket
    sock, context, tls_in_tls, server_hostname=server_hostname
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostna


100%|███████████████████████████████████████████████████████████████████████| 270151/270151 [16:55:08<00:00,  4.44it/s]
Exception in thread Thread-33:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 1336, in getresponse
    response.begin()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.

Exception in thread Thread-93:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 1336, in getresponse
    response.begin()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\PAC\Anaconda3\lib\socket.py", line 589, in readinto
    return sel

Exception in thread Thread-113:
Traceback (most recent call last):
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\PAC\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 1336, in getresponse
    response.begin()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\PAC\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\PAC\Anaconda3\lib\socket.py", line 589, in readinto
    return se

In [ ]:
json.dump(data, open('data.json', 'w'))